In [1]:
from stage import getStage, fullPrint
import pandas as pd
import datetime
from datetime import date, datetime
import yahoo_fin.stock_info as yf

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            #print(df.at[date,'MA30Slope'],df.at[date,'RS'])
            #!!!!!!!! or for new params
            #!!!!!!!!
            #!!!!!!!!
            #!!!!!!!!
            if  df.at[date,'MA30Slope'] > 0.9804811720459655 and df.loc[date,'RS'] > -0.19103234802670008:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1
    #print(goodSectorDf.at[date,'Sectors'])
fullPrint(goodSectorDf)

                                                      Sectors
1999-01-08                                                 {}
1999-01-15                                                 {}
1999-01-22                                                 {}
1999-01-29                                                 {}
1999-02-05                                                 {}
1999-02-12                                                 {}
1999-02-19                                                 {}
1999-02-26                                                 {}
1999-03-05                                                 {}
1999-03-12                                                 {}
1999-03-19                                                 {}
1999-03-26                                                 {}
1999-04-09                                                 {}
1999-04-16                                                 {}
1999-04-23                                                 {}
1999-04-

In [3]:
removeList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")

In [6]:
# Open a file
result = open("Testdata/Realtime/result"+date.today().strftime('%Y-%m-%d')+".txt", "w")
buyList = []
holdList = []
sellList = []
for symbol in nasdaqList:
    df = getStage(symbol,[0.5590154301570781, 1.0038092967383403, 0.38303606143798996, 1.0362389796691118, 1.0030905657509877, 1.0189883649147726, 0.8666655791915169, 0.9855913108128587, 1.3908949927299494, 0.9804811720459655, -0.19103234802670008, 0.9359023877540643, 0.9908752120306481, 0.808767804685719, 0.9841940289653691],goodSectorDf)
    shares = 0
    sharesSPY = 0
    if df.empty:
        continue
    if df.iloc[-1].Stage=="Buy":
        print(symbol+" buy " + df.iloc[-1].support)
        result.write(symbol+" buy " + df.iloc[-1].support + "\n")
        buyList.append(symbol)
    elif df.iloc[-1].Stage=="Stage 2":
        print(symbol+" hold " + df.iloc[-1].support)
        result.write(symbol+" hold " + df.iloc[-1].support + "\n")
        holdList.append(holdList)
    elif df.iloc[-1].Stage[0:4]=="Sell":
        print(symbol+" sell")
        result.write(symbol+" sell\n")
        sellList.append(sellList)
result.close()

ACET buy
AGRI sell
ARTW buy
CALM buy
CPLP hold
HBP hold
KROS buy
LAND buy
NFE buy
PNBK hold
SAMG hold
SGML buy
TCDA buy
TH buy
TWNK buy
UG buy


In [9]:
df = getStage("SAMG",[0.5590154301570781, 1.0038092967383403, 0.38303606143798996, 1.0362389796691118, 1.0030905657509877, 1.0189883649147726, 0.8666655791915169, 0.9855913108128587, 1.3908949927299494, 0.9804811720459655, -0.19103234802670008, 0.9359023877540643, 0.9908752120306481, 0.808767804685719, 0.9841940289653691],goodSectorDf)
df.to_csv("SAMG.csv")